In [1]:
################################ IMPORTS ###################################
import os
import sys
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib.dates import YearLocator, MonthLocator, DayLocator, WeekdayLocator, DateFormatter
from dateutil.relativedelta import relativedelta
### LINEAR REGRETION
from pyspark.ml.regression import LinearRegression
### RANDOM FOREST
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator



In [2]:
SpSession = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("BPSpark") \
    .config("spark.executor.memory", "1g") \
    .config("spark.cores.max","2") \
    .config("spark.sql.warehouse.dir", "file:///c:/temp/spark-warehouse")\
    .getOrCreate()

In [3]:
#Get the Spark Context from Spark Session    
SpContext = SpSession.sparkContext
try:
    sc = pyspark.SparkContext(appName = 'App')
    spark = SparkSession(sparkContext=sc)
    print ("SparkSession initialized")
except ValueError:
    print ("SparkSession already initialized")

SparkSession already initialized


In [23]:
################################ READ DATA ###################################
test2 = SpSession.read.option("delimiter", ";").option("header", "true").csv("D:/School/STAGE_BP/Stage_BachlerProef/datasets/jams2.csv")

In [24]:
test2.show(2)

+----------------+----+--------------+------+-----+-----+-----+--------+---------+------------+----------+--------------------+----------------+----+----+----+----+----+----+----+----+----+
|         PubTime|City|        Street|Length|Delay|Speed|Level|RoadType|StartNode|     EndNode|      Uuid|   BlockingAlertUuid|   RetrievalTime|_c13|_c14|_c15|_c16|_c17|_c18|_c19|_c20|_c21|
+----------------+----+--------------+------+-----+-----+-----+--------+---------+------------+----------+--------------------+----------------+----+----+----+----+----+----+----+----+----+
|31/10/2017 15:12| Eke|Kortebosstraat|   155|   -1|    0|    5|       1|     null|   Bosstraat|1103407095|e651a600-77c6-3f5...|11/11/2017 18:37|null|null|null|null|null|null|null|null|null|
|31/10/2017 15:12| Eke|Kortebosstraat|   255|   -1|    0|    5|       1|     null|Vijverstraat|1094427178|e43b1cea-4797-312...|11/11/2017 18:37|null|null|null|null|null|null|null|null|null|
+----------------+----+--------------+------+-----

In [29]:
test2.select('City').show(10,False)

+----+
|City|
+----+
|Eke |
|Eke |
|Eke |
|Eke |
|Eke |
|Eke |
|Eke |
|Eke |
|Eke |
|Eke |
+----+
only showing top 10 rows



In [30]:
################################ ML ###################################

# convert RDD to dataframe
ColNames = ["City","Speed"]
DFdataLR= test2(ColNames)
# split data in to training and test data
trainTestLR=DFdataLR.randomSplit([0.5,0.5])
trainDFLR= trainTestLR[0]
testDFLR= trainTestLR[1]
# LinearRegression model
ModelLR = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
# train model with data
ModelresultLR = ModelLR.fit(trainDFLR)
# predict with test data
PredictLR = ModelresultLR.transform(testDFLR).cashe()
# show prediction
predictionsLR = PredictLR.select("prediction").rdd.map(lambda x: x[0])
labels = fullPredictions.select("label").rdd.map(lambda x: x[0])
predictionzip=predictionsLR.zip(labels).collect()
for prediction in predictionzip:
    print(prediction)




TypeError: 'DataFrame' object is not callable

In [11]:
test = SpSession.read.json("D:/School/STAGE_BP/twitterBU/*.json")

In [ ]:
### RANDOM FOREST
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)

# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("predictedLabel", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

rfModel = model.stages[2]
print(rfModel)  # summary only